### Imports 

In [2]:
%pip install gurobipy

Note: you may need to restart the kernel to use updated packages.


In [9]:
import gurobipy as gb
import math
import re
from scipy.spatial import distance_matrix
import numpy as np

In [65]:
class PDSVRPModel:
    def __init__(self, C_T, C_D, D, h, distances, t_t, t_d, Q_t, Q_d, T_t, T_d, w):
        self.C_T = C_T #truck transportation cost
        self.C_D = C_D #drone transportation cost
        self.D = D #number of drones
        self.N = len(w) #number of nodes
        self.h = h #number of trucks
        self.distances= distances #matrix of distances
        self.t_t = t_t #matrix of trucks travel times
        self.t_d =t_d #vector of drones travel times
        self.Q_t = Q_t #truck capacity
        self.Q_d = Q_d #drone capacity
        self.T_t = T_t #max time truck
        self.T_d = T_d #max time drones
        self.w = w #weights vector
        
        self.model = gb.Model("OptimizationModel")
        self.x = None
        self.y = None
        self.z = None
        self.u = None
        
    def build_model(self):
        # Define the decision variables
        self.x = self.model.addVars([(i,j) for i in range(self.N) for j in range(self.N) if i!=j], vtype=gb.GRB.BINARY, name="x")#ho messo +1 perchè lo 0 è il depot e poi ci sono N clienti
        self.y = self.model.addVars([(i,k) for i in range(1, self.N) for k in range(self.D)], vtype=gb.GRB.BINARY, name="y")#ok
        self.z = self.model.addVars([(i,j) for i in range(self.N) for j in range(self.N) if i!=j], lb=0, ub=self.T_t, vtype=gb.GRB.CONTINUOUS, name="z")#ok (lb e ub sono già specificati nei constraits) 
        self.u = self.model.addVars([(i) for i in range(1, self.N)], lb=0, ub=self.Q_t, vtype=gb.GRB.CONTINUOUS, name="u")#ok

        # Define the objective function
        self.model.setObjective(
            gb.quicksum(self.C_T * self.distances[i,j] * self.x[i, j] for i in range (self.N) for j in range (self.N) if i != j) +
            gb.quicksum(self.C_D * self.distances[0,k] * self.y[k, l] for k in range (1, self.N) for l in range(self.D)),
            gb.GRB.MINIMIZE
        ) #ok

        N_t = [i for i in range(1,self.N) if self.w[i]> self.Q_d] #indexes of clients that must be served by trucks
        N_f= [i for i in range(1,self.N) if self.w[i]<= self.Q_d] #indexes of clients that can ber served either by e truck or a drone

        # Add constraints
        self.model.addConstr(gb.quicksum(self.x[0, i] for i in range (1, self.N)) <= self.h, "2) Constraint on number of trucks") #ok

        for j in range(self.N):
            self.model.addConstr(gb.quicksum(self.x[i, j] for i in range(self.N) if i != j) == gb.quicksum(self.x[j, i] for i in range(self.N) if i != j), "3) Flow constraint") #ok

        for j in N_f:
            self.model.addConstr(gb.quicksum(self.x[i, j] for i in range(self.N) if i != j) + gb.quicksum(self.y[j, k] for k in range(self.D)) == 1, "4) Every customer in N_f must be served") #ok

        for j in N_t:
            self.model.addConstr(gb.quicksum(self.x[i, j] for i in range(self.N) if i != j) == 1, "5) Truck customers must be visited by only trucks") #ok

        for i in range (1, self.N):
            for j in range (1, self.N):
                if j != i:
                    self.model.addConstr(self.u[i] - self.u[j] + self.Q_t * self.x[i, j] <= self.Q_t - self.w[j], "6) Miller-Tucker_Zemlin constraint") #ok

        for k in range(self.D):
            self.model.addConstr(gb.quicksum(self.y[j, k] * self.t_d[j] for j in N_f) <= self.T_d, "7) Time constraint for drones") #ok

        for i in range(1, self.N):
            self.model.addConstr(gb.quicksum(self.z[l, i] for l in range(self.N) if l != i) + gb.quicksum(self.t_t[i, j] * self.x[i, j] for j in range(self.N) if j != i) == gb.quicksum(self.z[i, j] for j in range(self.N) if j != i),"8) Inductive step for induction method of cumulative time additions")

        for i in range(1, self.N):
            self.model.addConstr(self.z[0, i] == self.t_t[0, i] * self.x[0, i], "9) Basic step for induction method of cumulative time additions")

        for i in range(1, self.N):
            self.model.addConstr(self.z[i, 0] <= self.T_t * self.x[i, 0], "10) Truck time limit constraint")
        '''
        self.model.addConstrs((self.x[i, j] == 0 for i )n self.N for j in self.N if i == j), "Vincolo(11)")
        self.model.addConstrs((self.y[j, k] == 0 for j in self.C_T for k in self.D), "Vincolo(12)")
        self.model.addConstrs((0 <= self.u[i] <= self.Q for i in self.C), "Vincolo(13)")
        self.model.addConstrs((0 <= self.z[i, j] <= self.T for i in self.N for j in self.N if i != j), "Vincolo(14)")
        '''
    def solve(self):
        self.model.optimize()

    def print_results(self):
        if self.model.status == gb.GRB.OPTIMAL:
            for v in self.model.getVars():
                print(f'{v.varName}: {v.x}')
            print(f'Obj: {self.model.objVal}')
        else:
            print("No optimal solution found.")

'''
# Esempio di utilizzo della classe
# Definire i dati del problema (questo deve essere adattato ai dati specifici)
C = ...
C_T = ...
D = ...
N = ...
h = ...
d_ij = ...
d_prime_jk = ...
t_ij = ...
t_0i = ...
Q = ...
Q_bar = ...
T = ...
T_prime = ...

# Creare un'istanza della classe
opt_model = OptimizationModel(C, C_T, D, N, h, d_ij, d_prime_jk, t_ij, t_0i, Q, Q_bar, T, T_prime)

# Costruire il modello
opt_model.build_model()

# Risolvere il modello
opt_model.solve()

# Stampare i risultati
opt_model.print_results()
'''

"\n# Esempio di utilizzo della classe\n# Definire i dati del problema (questo deve essere adattato ai dati specifici)\nC = ...\nC_T = ...\nD = ...\nN = ...\nh = ...\nd_ij = ...\nd_prime_jk = ...\nt_ij = ...\nt_0i = ...\nQ = ...\nQ_bar = ...\nT = ...\nT_prime = ...\n\n# Creare un'istanza della classe\nopt_model = OptimizationModel(C, C_T, D, N, h, d_ij, d_prime_jk, t_ij, t_0i, Q, Q_bar, T, T_prime)\n\n# Costruire il modello\nopt_model.build_model()\n\n# Risolvere il modello\nopt_model.solve()\n\n# Stampare i risultati\nopt_model.print_results()\n"

In [60]:
def load_instance(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()
    
    # Dictionary of values
    values = {}
    
    # Lists to save coordinates and weights
    coordinates = []
    weights = []

    for line in lines:
        if ',' in line:
            key, value = line.strip().split(',')
            if key in ["NUM DRONES", "NUM TRUCKS"]:
                values[key] = int(value)
            elif key in ["TRUCK CAP", "DRONE CAP", "TRUCK SPEED", "DRONE SPEED", "DRONE ENDURANCE", "DRONE TIME LIMIT", "TRUCK TIME LIMIT", "TRUCK UNIT COST", "DRONE UNIT COST"]:
                values[key] = float(value)
        else:
            parts = re.sub(' +',' ', line).split()[1:] 
            # Aggiungi le coordinate (x, y) e il peso alla lista corrispondente
            x, y, weight = map(float, parts)
            coordinates.append((x, y))
            weights.append(weight)

    return values, coordinates, weights

# Utilizzo della funzione
file_path = "./instances/test_instance.txt"  # Sostituisci con il percorso corretto del tuo file
values, coordinates, weights = load_instance(file_path)

# Stampa i valori per verificare
print("Values:", values)
print("Coordinate:")
for coord in coordinates:
    print(coord)

print("Pesi:")
for weight in weights:
    print(weight)



Values: {'NUM DRONES': 4, 'NUM TRUCKS': 10, 'TRUCK CAP': 1300.0, 'DRONE CAP': 2.27, 'TRUCK SPEED': 30.0, 'DRONE SPEED': 40.0, 'DRONE ENDURANCE': 0.6, 'DRONE TIME LIMIT': 1.5, 'TRUCK TIME LIMIT': 1.5, 'TRUCK UNIT COST': 1.25, 'DRONE UNIT COST': 0.03}
Coordinate:
(10.0, 10.0)
(8.08, 0.81)
(10.07, 16.46)
(8.15, 10.68)
(4.47, 3.39)
(13.91, 12.49)
(13.75, 19.07)
(1.87, 10.74)
(1.88, 14.08)
(17.35, 15.64)
(17.21, 16.95)
(17.8, 2.38)
(8.02, 4.22)
(13.19, 7.99)
(7.04, 16.33)
(3.92, 10.07)
(12.62, 4.54)
(9.75, 6.97)
(15.02, 11.41)
(15.19, 4.17)
(10.72, 13.63)
(12.64, 4.36)
(13.13, 4.39)
(12.67, 3.33)
(10.47, 8.7)
(9.35, 6.36)
(8.72, 8.69)
(15.0, 11.18)
(14.86, 4.33)
(10.71, 13.65)
(10.77, 14.5)
Pesi:
0.0
1.9
1.02
0.18
0.93
1.04
48.34
1.82
0.98
1.37
1.04
18.2
39.0
0.98
1.85
0.62
0.86
1.51
0.36
2.21
1.44
1.9
0.94
1.77
0.86
1.88
1.78
0.87
1.11
30.4
0.63


In [61]:
def distance_matrix_computation(coord):
    coord_array = np.array(coord)
    return distance_matrix(coord_array, coord_array)

In [62]:
d_matrix= distance_matrix_computation(coordinates)
t_t_matrix = d_matrix/values["TRUCK SPEED"]
t_d_vector = d_matrix[0]/values["DRONE SPEED"]

In [66]:
model = PDSVRPModel(C_T= values["TRUCK UNIT COST"], 
                    C_D=values["DRONE UNIT COST"],
                    D=values["NUM DRONES"],
                    h=values["NUM TRUCKS"],
                    distances= d_matrix,
                    t_t= t_t_matrix,
                    t_d= t_d_vector,
                    Q_t= values["TRUCK CAP"],
                    Q_d= values["DRONE CAP"],
                    T_t= values["TRUCK TIME LIMIT"],
                    T_d= values["DRONE TIME LIMIT"],
                    w=weights)

In [67]:
model.build_model()

In [68]:
model.solve()


Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (win64 - Windows 11.0 (22631.2))

CPU model: 11th Gen Intel(R) Core(TM) i7-1165G7 @ 2.80GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads



GurobiError: Model too large for size-limited license; visit https://gurobi.com/unrestricted for more information